# ELM

## Importing Libraries

In [6]:
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os

# Dateset Class and DataLoader

In [2]:
class ELM_Dataset(Dataset):
    def __init__(self, features, labels):
        super(ELM_Dataset, self).__init__()

        self.features = torch.Tensor(features, dtype = torch.float32)
        self.labels = torch.Tensor(labels, dtype = torch.float32)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]

        return feature, label

## Importing Dataset

In [5]:
dataset_list = os.listdir(path = '../Datasets')
print(dataset_list)

['MEFAR_DOWN.csv', 'MEFAR_MID.csv', 'MEFAR_UP.csv']


## ELM Class

In [ ]:
class ELM(nn.Module):
    def __init__(self, input_size, hidden_neurons):
        super(ELM, self).__init__()

        self.input_size = input_size
        self.hidden_neurons = hidden_neurons

        #Model Layers
        self.hidden_layer = nn.Linear(input_size, hidden_neurons)
        self.output_layer = nn.Linear(hidden_neurons, 1)

        W = torch.rand((input_size, hidden_neurons))
        b = torch.rand((1, hidden_neurons))

    def forward(self, x):
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        x = nn.Sigmoid(x)

        return x

In [19]:
X_values = elm_data.iloc[:, :-1].values
y_values = elm_data.iloc[:, -1].values